##Connecting to Hopsworks Feature Store

In [ ]:
!pip install hopsworks

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

In [ ]:
air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality_fg',
    version = 2
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather_fg',
    version = 1
)

In [ ]:
query = air_quality_fg.select_all().join(weather_fg.select_all())

In [ ]:
query.read()

## Feature View Creation and Retrieving

In [ ]:
query = air_quality_fg.select_all().join(weather_fg.select_all())

query_show = query.show(5)
col_names = query_show.columns

query_show

### Transformation functions

In [ ]:
[t_func.name for t_func in fs.get_transformation_functions()]

In [ ]:
category_cols = ['date','weathercode','sunrise','sunset','aqi']

mapping_transformers = {col_name:fs.get_transformation_function(name='standard_scaler') for col_name in col_names if col_name not in category_cols}
category_cols = {col_name:fs.get_transformation_function(name='label_encoder') for col_name in category_cols if col_name not in ['date','aqi']}

mapping_transformers.update(category_cols)

## Creating Feature View

In [ ]:
feature_view = fs.create_feature_view(
    name = 'air_quality_fv',
    version = 1,
    labels=["aqi"],
    query = query
)